# Document

### 1. Convert Docs to PDF

**Convert .doc to .docx**

In [121]:
from glob import glob
import re
import os
import win32com.client as win32
from win32com.client import constants

def save_as_docx(path):
    # Opening MS Word
    word = win32.gencache.EnsureDispatch('Word.Application')
    doc = word.Documents.Open(path)
    doc.Activate ()

    # Rename path with .docx
    new_file_abs = os.path.abspath(path)
    new_file_abs = re.sub(r'\.\w+$', '.docx', new_file_abs)

    # Save and Close
    word.ActiveDocument.SaveAs(
        new_file_abs, FileFormat=constants.wdFormatXMLDocument
    )
    doc.Close(False)

In [123]:
save_as_docx('D:\Workplaces\Python\ComputerVision\PdfAutoSign\docs\To trinh test VBNB 2 PB trinh xin phe duyet 1 LD.doc')

**Delete redundant words**

In [127]:
import re
from docx import Document

def docx_replace_regex(doc_obj, regex , replace, a=False):

    for p in doc_obj.paragraphs:
        if regex.search(p.text):
            text = regex.sub(replace, p.text)
            p.text = text

    for table in doc_obj.tables:
        for row in table.rows:
            for cell in row.cells:
                docx_replace_regex(cell, regex , replace, True)

In [48]:
filename = "D:\Workplaces\Python\ComputerVision\PdfAutoSign\docs\Cong van test VBNB giua 2 PB_LMH.docx"
doc = Document(filename)

s1 = r"Số:.{0,20}/\w{2,10}"
regex1 = re.compile(s1)
replace1 = " " * 10 + '\n' + " " * 10
docx_replace_regex(doc, regex1 , replace1)

s2 = r"Hà Nội.{0,10}ngày.{0,10}tháng.{0,10}năm"
regex2 = re.compile(s2)
replace2 = " " * 10 + '\n' + " " * 10
docx_replace_regex(doc, regex2 , replace2)

doc.save('file.docx')

**Convert to PDF**

In [133]:
from docx2pdf import convert
convert('D:\Workplaces\Python\ComputerVision\PdfAutoSign\docs\To trinh test VBNB 2 PB trinh xin phe duyet 1 LD.docx', 'pdfs/result2.pdf')









100%|██████████| 1/1 [00:00<00:00,  1.26it/s]


**Find word coordinate**

In [103]:
from pdfminer.layout import LAParams, LTTextBox
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator

def find_coordinates(file_path, _text):
    # Open file
    fp = open(file_path, 'rb')
    
    # Init PDF reader
    rsrcmgr = PDFResourceManager()
    laparams = LAParams()
    device = PDFPageAggregator(rsrcmgr, laparams=laparams)
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    pages = PDFPage.get_pages(fp)

    # Loop through pages
    for i, page in enumerate(pages):
        print('Processing page: {}'.format(i))
        interpreter.process_page(page)
        layout = device.get_result()
        
        for lobj in layout:
            if isinstance(lobj, LTTextBox):
                coords, text = lobj.bbox[:4], lobj.get_text()
                if _text in text:
                    print('Found at page %d %r: %s' % (i, coords, text))
                    return i, coords, text
                
    return None, None, None                

In [134]:
page_num, coords, text = find_coordinates('pdfs/result2.pdf', 'Lê Minh Hoàng')

Processing page: 0
Found at page 0 (212.09, 320.17064000000005, 351.07, 331.96424): PGS.TS Lê Minh Hoàng 



In [135]:
import fitz

def draw_rect(file_path, page_num, coords):
    doc = fitz.open(file_path)
    page = doc[page_num]
    
    page.draw_rect(
        [
            coords[2], page.mediabox[3] - coords[3],   # top-right
            coords[2] + 100, page.mediabox[3] - coords[3] + 100    # bottom-right
        ],  
        color = (0, 1, 0), 
        width = 2
    )
    
    # Save pdf
    doc.save('result.pdf')

In [137]:
draw_rect('pdfs/result2.pdf', page_num, coords)

Rect(0.0, 0.0, 595.32, 841.92)


In [61]:
def small_sign(file, text):
    fp = open(file, 'rb')

    rsrcmgr = PDFResourceManager()
    laparams = LAParams()
    device = PDFPageAggregator(rsrcmgr, laparams=laparams)
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    pages = PDFPage.get_pages(fp)

    for (i, page) in enumerate(pages):
        interpreter.process_page(page)
        layout = device.get_result()
        for lobj in layout:
            if isinstance(lobj, LTTextBox) and text in lobj.get_text():
                page_num = i

                for line in lobj:
                    if text in line.get_text():
                        x0, y0_orig, x1, y1_orig = line.bbox[0], line.bbox[1], line.bbox[2], line.bbox[3]
                        text = line.get_text()
                        draw_rect(file, x1, y1_orig, page_num)
                        print('%r: %s' % ((x0, y1_orig, x1, y0_orig), text))
    fp.close()